In [22]:
from pydantic import BaseModel, Field

from voice_agent_flow.agents import AgentSession
from voice_agent_flow.agents.agent_node import AgentNode, DoHangUp, HangUpNode
from voice_agent_flow.agents.multi_agent_runner import MultiAgentRunner
from voice_agent_flow.llms import create_pydantic_azure_openai
from typing import Optional


class CustomerName(BaseModel):
    
    """Greet the customer with the customer's name in the greeting message. If not explicitly told that the name is wrong or wrong number dialed, assume the name is confirmed.
    Example: 
        Customer Service Representative: "喂，您好，请问是xxx吗?"
        Customer: "/呃/你说/是的/哪里/咋了/哦/是/嗯...."  -> create CustomerName(name="xxx") # all the ambiguous response should be treated as confirmation
        Customer: "/不是/打错了/我不是" -> -> create CustomerName(name=None, checked=False)
        Customer: "用户无法接听, 请在语音信箱留言" -> create CustomerName(name = None, checked = False)
        Customer: "您好，我是xxx的智能助手，..." -> create CustomerName(name = 'xxx', checked = False)
    """
    name: Optional[str] = Field(None, description = 'The name of the customer')
    checked: Optional[bool] = Field(True, description = 'Whether the customer name is confirmed(not explicitly rejected)')
    
    def transfer(self) -> str:
        
        if self.checked is None or self.checked:
            print("Transferring to FinancialSupportStatus")
            return 'financial_support_inquiry'
        
        else:
            print("Transferring to HangUp")
            return 'hangup'
    
    
class FinancialSupportStatus(BaseModel):
    
    """Whether the customer need financial support. True is the customer do not refuse/reject financial support.
    As long the the customer asking product details or show insterest, create FinancialSupportInquiry(require_support=True).
    Explicit rejection or refusal should create FinancialSupportStatus(require_support=False).
    Other case, you should continue to ask the customer for clarification.
    """
    require_support: bool = Field(..., description = 'Whether the customer need financial support')
    
    def transfer(self) -> str:
        
        if self.require_support:
            print("Transferring to VehicleNotUnderRepayment")
            return 'vehicle_payment_status'
        
        else:
            print("Transferring to HangUp")
            return 'hangup'
    
    
class PaymentMethod(BaseModel):
    """
    Whether the vehicle is under a auto loan or fully paid off.
    It the vehicle is not bought with a loan, is_not_under_repayment = True.
    If the vehicle is bought with a loan, and there are ongoing installments currently, is_not_under_repayment = False.
    """
    
    is_not_under_repayment: bool = Field(..., description = 'Whether the vehicle is not under repayment')
    
    def transfer(self) -> str:
        
        if self.is_not_under_repayment:
            print("Transferring to VehicleLiscenceUnderControl")
            return 'vehicle_liscence_under_control'
        
        else:
            print("Transferring to HangUp")
            return 'hangup'
    
class VehicleLiscenceUnderControl(BaseModel):
    """Whether the vehicle liscence(Chinese机动车行驶证， 绿本， 大本) is under the customer's control.
    If the vehicle liscense is under the customer's control, create VehicleLiscenceUnderControl(is_under_control=False).
    If the vehicle liscense is under the customer's spouse or family member's control, create VehicleLiscenceUnderControl(is_under_control=True).
    If the vehicle liscense ir under his/her company's control, create VehicleLiscenceUnderControl(is_under_control=False).
    """
    is_under_control: bool = Field(..., description = 'Whether the vehicle liscence is under the customer\'s control')
    
    def transfer(self) -> str:
        
        if self.is_under_control:
            print("Transferring to AgreeToAddWechatAccount")
            return "agree_wechat_add"
        
        else:
            print("Transferring to HangUp")
            return 'hangup'


class AgreeToAddWechatAccount(BaseModel):
    """Whether the customer agree to add wechat account for further contact.
    If the customer agree to add wechat account, create AgreeToAddWechatAccount(agree=True).
    If the customer refuse to add wechat account, create AgreeToAddWechatAccount(agree=False).
    """
    agree: bool = Field(..., description = 'Whether the customer agree to add wechat account')
    
    def transfer(self) -> str:
        print("Transferring to WeChatId")
        return "ask_wechat_id"
    
    
class WeChatId(BaseModel):
    """The WeChat ID provided by the customer for further contact.
    Example:
        Customer Service Representative: "请问您的微信号是多少？"
        Customer: "我的微信号是abc123" -> create WeChatId(wechat_id="abc123")
    """
    wechat_id: str = Field(..., description = 'The WeChat ID provided by the customer')
    
    def transfer(self) -> str:
        print("Transferring to HangUp")
        return 'hangup'

In [ ]:
model = create_pydantic_azure_openai('gpt-4o-mini')

instruction = """
You are a customer service representative in a auto finance company. You task is to talk with customer via telephone to collection information.
You will be given one task defined by a schema, you should talk with the user until you can fill the schema completely.
You shoule speak in Chinese.
On each text reply, you should output as less text as possible to collect information.

Overall Conversation Policy: For yes or not question, if the customer's response is not explicitly reject or refuse, you should assume the customer agree or accept it.
Then create the schema with the corresponding fields filled.

Note you are part of a multi-agent system, do not add additional explaination, greeting or closing statement, just focus on collecting information to fill the schema.
Do not Generate Structured output until you have collected information defined by the schema. Before that ,you shoule chat with the customer.

In each step, you can do 2 things: 
    1) try to collect information defined in the current step.
    2) Handle customer's question or concern, but keep it brief and concise.
    
Do not ask extra questions that are not metioned in the step instruction.

Style Instruction:
    1. Output in Chinese.
    2. Output in colloquial style, make the conversation fluent by enriching the examples. 
    3. Add filler words properly to make the conversation more nature and fluent.

"""

agents = {
    
    # Complex business rules, you need more prompt, but just in this step.
    "customer_name_inquiry": AgentNode(
        name="customer_name_inquiry",
        model=model,
        instruction=instruction,
        task_cls= CustomerName,
        step_instruction=(
            "Confirm the customer's name with a greeting message(customer name included in the message)."
            "Amubiguous response from customer should be treated as confirmation and create the schema. Current Customer Name: 李老三"
            "Any response for the message will be treated as confirmation unless the customer explicitly says he/she is not the person or dialed wrong number."
            "Event a simple ‘嗯’, ‘呃’, '哪里'，‘你说’ indicates a confirmation. You shoule create the schema immediately"
        ),
            
        examples=["您好，请问是xxx(plug customer name here)吗？"],
        ),
    
    # Complex business rules, you need more prompt, but just in this step.
    "financial_support_inquiry": AgentNode(
        name="financial_support_inquiry",
        model=model,
        instruction=instruction,
        task_cls= FinancialSupportStatus,
        step_instruction=(
            "Briefly introduce yourself(您好，这边是易鑫集团的金融顾问) and ask the customer whether he/she needs financial support.（Do COPY the example）. "
            "You only need to collect a yes/no question in this step, if the customer asking other questions, this usually indicates she/he has financial support requirement."
            "You should create FinancialSupportStatus if the customer does not reject/refuse this question."
        ),
        examples=["您好，这边是易鑫集团的金融顾问，看到你的申请的资金方案，您最近是有资金需求吗？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "vehicle_payment_status": AgentNode(
        name="vehicle_payment_status",
        model=model,
        instruction=instruction,
        task_cls= PaymentMethod,
        step_instruction=(
            "Start a question by asking the customer if his/her vehicle is bought on finance or fully paid off.(您的车是全款买的还是按揭买的？)."
            "If the vehicle is fully paid off（全款）, create VehicleNotUnderRepayment(is_not_under_repayment=True). "
            "If the vehicle is bought with a loan(按揭)， follow up by asking '那您的分期现在还完了么'."
            "If the user confirms like 还清了/换完了, create VehicleNotUnderRepayment(is_not_under_repayment=True)."
            "If the vehicle is still under repayment (还没有/还在贷款/贷款呢/还差一点/过几个月/还有几期), create VehicleNotUnderRepayment(is_not_under_repayment=False)."
            "Overall, you if there are ongoing installments, is_not_under_repayment = False, else True"
        ),
        examples=["请问您的车辆目前是已经还清贷款了吗？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "vehicle_liscence_under_control": AgentNode(
        name="vehicle_liscence_under_control",
        model=model,
        instruction=instruction,
        task_cls= VehicleLiscenceUnderControl,
        step_instruction=(
            "Ask the customer whether the vehicle liscence is under the customer's control."
            "If the customer says the vehicle liscense is under his/her control like '在手里', '在家里'，'嗯', create VehicleLiscenceUnderControl(is_under_control=True)."
            "If the vehicle liscense is under his/her spouse or family member's control, create VehicleLiscenceUnderControl(is_under_control=True)."
            "If the vehicle liscense ir under his/her company's control, create VehicleLiscenceUnderControl(is_under_control=False)."
        ),
        examples=["请问您的车辆行驶证现在是在您本人手上吗？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "agree_wechat_add": AgentNode(
        name="agree_wechat_add",
        model=model,
        instruction=instruction,
        step_instruction="Ask the customer whether he/she agree to add wechat account for further contact.",
        task_cls= AgreeToAddWechatAccount,
        examples=["我们这边后续办理业务需要添加一下您微信，您是否同意添加我们的微信账号？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "ask_wechat_id": AgentNode(
        name="ask_wechat_id",
        model=model,
        instruction=instruction,
        step_instruction="Ask the customer for his/her wechat id.",
        task_cls= WeChatId,
        examples=["请问您的微信号是多少？"],
        ),
    
    "hangup": HangUpNode(model = model)
}

In [40]:
runner = MultiAgentRunner(
    agents=agents, 
    entry_agent_name="customer_name_inquiry",
    ending_message="好的，我们稍后会加您的微信，请你注意在服务通知后查看我们的企业微信请求，再见！"
)  

chat = AgentSession(runner)

In [41]:
_ = await chat.chat("喂")

🤖[customer_name_inquiry]...Working.
您好，请问是李老三吗？

In [42]:
_ = await chat.chat("我是他的语音助手，您跟我说一下找他什么事儿啊")

🤖[customer_name_inquiry]...Working.
Transferring to HangUp
AgentHandoff(status=None, message={'source_agent_name': 'customer_name_inquiry', 'target_agent_name': 'hangup'})
感谢您的协助！我只是想通知李老三有个重要的会议需要参加。如果您能帮我转达一下，我将非常感谢。祝您生活愉快，再见。

In [30]:
_ = await chat.chat("是的")

🤖[customer_name_inquiry]...Working.
Transferring to FinancialSupportStatus
AgentHandoff(status=None, message={'source_agent_name': 'customer_name_inquiry', 'target_agent_name': 'financial_support_inquiry'})
您好，这边是易鑫集团的金融顾问，看到你的申请的资金方案，您最近是有资金需求吗？

In [31]:
_ = await chat.chat("对")

🤖[financial_support_inquiry]...Working.
Transferring to VehicleNotUnderRepayment
AgentHandoff(status=None, message={'source_agent_name': 'financial_support_inquiry', 'target_agent_name': 'vehicle_payment_status'})
请问您目前的车辆是全款买的还是按揭买的？

In [32]:
_ = await chat.chat("什么？")

🤖[vehicle_payment_status]...Working.
您的车是全款买的，还是通过贷款（按揭）买的呢？

In [33]:
_ = await chat.chat("贷款的")

🤖[vehicle_payment_status]...Working.
那您的分期现在还完了么？

In [34]:
_ = await chat.chat("还清了")

🤖[vehicle_payment_status]...Working.
Transferring to VehicleLiscenceUnderControl
AgentHandoff(status=None, message={'source_agent_name': 'vehicle_payment_status', 'target_agent_name': 'vehicle_liscence_under_control'})
好的，很棒！请问您的车辆行驶证现在是在您本人手上吗？

In [35]:
_ = await chat.chat("在我妈妈手里")

🤖[vehicle_liscence_under_control]...Working.
Transferring to AgreeToAddWechatAccount
AgentHandoff(status=None, message={'source_agent_name': 'vehicle_liscence_under_control', 'target_agent_name': 'agree_wechat_add'})
明白了，没问题！那接下来，我需要问您是否同意添加我们的微信账号以便后续联系？

In [36]:
_ = await chat.chat("嗯")

🤖[agree_wechat_add]...Working.
Transferring to WeChatId
AgentHandoff(status=None, message={'source_agent_name': 'agree_wechat_add', 'target_agent_name': 'ask_wechat_id'})
请问您的微信号是多少呢？

In [37]:
_ = await chat.chat("15001395923")

🤖[ask_wechat_id]...Working.
Transferring to HangUp
AgentHandoff(status=None, message={'source_agent_name': 'ask_wechat_id', 'target_agent_name': 'hangup'})
非常感谢您提供的信息。如果您没有其他问题，我将结束今天的通话。祝您生活愉快，再见！

In [38]:
_ = await chat.chat("再见")

🤖[hangup]...Working.
HangupSignal(status=None, message=DoHangUp())
Conversation Ended with Hangup Signal.


In [39]:
chat.state

{'name': '李老三',
 'checked': True,
 'require_support': True,
 'is_not_under_repayment': True,
 'is_under_control': True,
 'agree': True,
 'wechat_id': '15001395923'}